In [1]:
import requests
def connected_to_internet(url='http://www.google.com/', timeout=5):
    try:
        _ = requests.get(url, timeout=timeout)
        return True
    except requests.ConnectionError:
        print("No internet connection available.")
    return False

connected_to_internet()

True

In [2]:
#!pip install -r requirements.txt
#!pip install fxcmpy
#!pip install forexconnect
#!pip install sklearn

In [3]:
import os
import threading
import time
import importlib
import sys
mpath = os.getcwd() + '/lib'
sys.path.append(mpath)

In [5]:
#from forexconnect import fxcorepy
import fxcmpyi
from sklearn import *
import pandas as pd

import SharedObjects
import pairs
importlib.reload(SharedObjects)
importlib.reload(fxcmpyi)
importlib.reload(pairs)
#importlib.reload(sklearn)

import numpy as np

In [6]:
#fxcmpy.__version__

minutes: m1, m5, m15 and m30,
hours: H1, H2, H3, H4, H6 and H8,
one day: D1,
one week: W1,
one month: M1.

In [7]:
#Set variables
T = 'm1' #Period
sz = 300; #Size
prs = dict()

Get access token from https://tradingstation.fxcm.com.

demo - for account type demo

real - for account type real

In [8]:
con = None
def custom():
    global con
    try:
        def login():
            global con
            TOKEN = '2720337434ea557dea25bad4cd9ad392d70f620d'
            con = fxcmpyi.fxcmpy(access_token=TOKEN, log_level='error', server='demo')
        threading.Thread(target=login).start()
    except:
        custom()
custom()
#con.close()

In [14]:
if con is not None:
    print(con.is_connected())
    shd_obj = SharedObjects.SharedObjects(con)

True


In [15]:
#Function to update data
def price_update(data, dataframe):
    symbol = data['Symbol']
    #print(symbol)
    #if( (symbol in prs) == True):
    #    for key, val in prs.items():
    #        if(val.sym == symbol):
    #            print(val.update(dataframe))
    #            break

In [16]:
try:
    def get_past_prices():
        for symbol in shd_obj.tradable_pairs:
            dtf = shd_obj.historical_data(symbol, T, sz)
            if symbol not in prs:
                prs[symbol] = pairs.Pair(symbol, dtf)
                shd_obj.stream_data(symbol, (price_update, prs.get(symbol).on_price_update) )
            
    threading.Thread(target=get_past_prices).start()
except:
    print("Error: unable to start thread")

In [81]:
#shd_obj.unstream_data()

In [82]:
#con.close()

In [ ]:
#status = shd_obj.get_status()
#if(status == 'You are logged in'):
    #shd_obj.stream_data(T, sz, price_update)

In [ ]:
##Reverses the dataframe rows
#df[::-1]

#c = df.drop(columns=['tickvolume'])
#dates[0].minute
#dates[0].hour

#con.is_subscribed(instruments[0])
#con.get_subscribed_symbols()

#con.unsubscribe_market_data('EUR/USD')
#print(shd_obj.pairs)
#con.close()

In [ ]:
#con.get_default_account()
#con.get_prices('EUR/USD')

In [ ]:
#daf = pd.DataFrame()
#symbol = ''
#def print_data(data, dataframe):
#    global daf
#    global symbol
#    symbol = data['Symbol']
#    daf = dataframe
#    print('%3d | %s | %s, %s, %s, %s, %s'
#          % (len(dataframe), data['Symbol'],
#             pd.to_datetime(int(data['Updated']), unit='ms'),
#             data['Rates'][0], data['Rates'][1], data['Rates'][2],
#             data['Rates'][3]))
    #print(daf)

In [ ]:
#import inspect
#source_file_DF = inspect.getsource(pd.concat)
#print(source_file_DF)

In [79]:
dtf = list()

lm = linear_model.LinearRegression()
for key in prs:
    keys = set(prs.keys())
    excludes = set([key])
    diff = keys.difference(excludes)
    x = pd.DataFrame()
    y = prs.get(key).prices['Close'].tolist()

    for ky in diff:
        x[ky] = prs.get(ky).prices['Close'].tolist()
    model = lm.fit(x,y)
    
    #R squared
    rs = lm.score(x,y)
    #Coefficients
    bs = model.coef_.tolist()
    #Intercept
    intercept = model.intercept_
    #Prediction (first)
    pred = lm.predict(x)[0]

    dez = {}
    if(y[len(y)-1] > pred):
        dez = {True: 'SELL', False: 'BUY'}
    elif(y[len(y)-1] < pred):
        dez = {True: 'BUY', False: 'SELL'}
    dez[1.0] = dez[True]
    dez[-1.0] = dez[False] 

    df = pd.DataFrame(index = [key, dez[True]])
    df['R_Squared'] = [rs, '']
    #df['Current'] = [y[len(y)-1], '-' ]
    df['Prediction'] = [pred, '' ]
    df['Intercept'] = [intercept, '-']
    hd = x.columns.tolist()
    for s in hd:
        c = bs[hd.index(s)]
        sgn = np.sign(c)
        df[s] = [c, dez[sgn] ]

    dtf.append(df)

In [80]:
dtf[1]

,R_Squared,Prediction,Intercept,GBP/USD,USD/CAD,Copper,AUD/USD,ZAR/JPY,USD/ZAR,NZD/USD,EUR/GBP,USD/CHF,EUR/USD
USD/JPY,0.96447,110.747,-48.7902,-30.4856,-2.50146,-0.637445,-0.644663,13.5785,6.9286,6.51602,-43.4836,-1.14502,31.146
BUY,,,-,SELL,SELL,SELL,SELL,BUY,BUY,BUY,SELL,SELL,BUY
